In [6]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#"tokenization" (the act of splitting the string into discrete "tokens")
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


.data\imdb\aclImdb_v1.tar.gz: 100%|███████████████████████████████████████████████| 84.1M/84.1M [00:19<00:00, 4.26MB/s]


In [17]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of testing examples: 25000


In [19]:
print(vars(train_data.examples[0]))

{'text': ['This', 'movie', 'has', 'got', 'to', 'be', 'one', 'of', 'the', 'worst', 'I', 'have', 'ever', 'seen', 'make', 'it', 'to', 'DVD', '!', '!', '!', 'The', 'story', 'line', 'might', 'have', 'clicked', 'if', 'the', 'film', 'had', 'more', 'funding', 'and', 'writers', 'that', 'would', 'have', 'cut', 'the', 'nonsense', 'and', 'sickly', 'scenes', 'that', 'I', 'highly', 'caution', 'parents', 'on', '....', 'But', 'the', 'story', 'line', 'is', 'like', 'a', 'loose', 'cannon', '.', 'If', 'there', 'was', 'such', 'a', 'thing', 'as', 'a', 'drive', 'thru', 'movie', 'maker', '-', 'this', 'one', 'would', 'have', 'sprung', 'from', 'that', '.', 'It', 'reminded', 'me', 'a', 'lot', 'of', 'the', 'quickie', 'films', 'that', 'were', 'put', 'out', 'in', 'the', '1960', "'s", ',', 'poor', 'script', 'writing', 'and', 'filming', '.', '<', 'br', '/><br', '/>The', 'only', 'sensible', 'characters', 'in', 'the', 'whole', 'movie', 'was', 'the', 'bartender', 'and', 'beaver', '.', 'The', 'rest', 'of', 'the', 'film',

In [14]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [16]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [20]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [22]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [23]:
print(TEXT.vocab.freqs.most_common(20))


[('the', 203566), (',', 192495), ('.', 165612), ('and', 109442), ('a', 109116), ('of', 100702), ('to', 93766), ('is', 76328), ('in', 61255), ('I', 54004), ('it', 53508), ('that', 49187), ('"', 44282), ("'s", 43329), ('this', 42445), ('-', 36690), ('/><br', 35752), ('was', 35034), ('as', 30384), ('with', 29774)]


In [24]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [25]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [26]:
BATCH_SIZE = 64

device = torch.device('cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [27]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [28]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [29]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [30]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [31]:
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [32]:
def binary_accuracy(preds, y):
    #Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [33]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [34]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [35]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [39]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 10m 23s
	Train Loss: 0.693 | Train Acc: 50.42%
	 Val. Loss: 0.697 |  Val. Acc: 49.74%
Epoch: 02 | Epoch Time: 9m 29s
	Train Loss: 0.693 | Train Acc: 50.55%
	 Val. Loss: 0.697 |  Val. Acc: 49.93%
Epoch: 03 | Epoch Time: 9m 5s
	Train Loss: 0.693 | Train Acc: 50.41%
	 Val. Loss: 0.697 |  Val. Acc: 49.64%
Epoch: 04 | Epoch Time: 8m 43s
	Train Loss: 0.693 | Train Acc: 50.38%
	 Val. Loss: 0.697 |  Val. Acc: 49.70%
Epoch: 05 | Epoch Time: 9m 11s
	Train Loss: 0.693 | Train Acc: 50.18%
	 Val. Loss: 0.697 |  Val. Acc: 49.96%


In [41]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.708 | Test Acc: 48.17%
